**Libs**:  
Scrapy | cartodb | numpy/pandas/matplotlib/scikit-learn | scikit-image | nltk | (plotly/graphene/etc)  

**Tools**:  
Python | firefox+firebug+firepath | Linux+CLI  

**Pages**:  
Idealista | Segundamano | ?  

**Time**:  
1h50m + 10m (Q/A)  

**Demos**:  
Scrapy | XPATH | panda | matplotlib | S.GIS (Filtering/etc) | CLI |  

# Siri, ¡Búscame un piso!
### Cómo encontrar el piso perfecto en ~2h

Yuri Numerov (Achifaifa)  
[achi.se](http://achi.se)  -  [achi (at) hush (.) ai]()

Euskal Encounter 24, 2015-07-XX

0) Introducción

1) Spam

A) Manera tradicional de buscar piso

B) Manera guay de buscar un piso

Introducción a las herramientas

Idea general

01) Localizando páginas web

02) Scrapy y XPATH

03) Escribiendo una araña

04) Aclarar y repetir para el resto de webs

05) Introducción a datos en CSV

06) Introducción a GIS (Filtrando por CP)

07) Primera visualización (Introducción a cartodb)

08) Datos sucios (Completando GPS)

09) Datos conseguidos. ¿Qué buscamos? (Piscina, jardín, precio...)

10) Asignando pesos a variables

11) Pandas, numpy, matplotlib: Primera tentativa

12) Introducción a machine learning (Datos de tecreros)

13) Resultados, mejora, etc

Conclusiones

Consejos, fuentes, información extra, etc

Dudas, preguntas

Despedida